# 在MNIST数据处理中加入注意力层

In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, Dense, Multiply
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from keras import Model

### 1.加载数据

In [2]:
((x_train, y_train), (x_test, y_test)) = mnist.load_data()
x_train.shape

### 2.数据变换

In [3]:
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255.0
x_test = x_test/255.0

y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test, 10)

### 3.构建模型

In [4]:
#输入层
inputs=Input(shape=(784,))
#w
probs=Dense(784,activation='softmax')(inputs)
mul=Multiply()([inputs,probs])
#fc层
mul=Dense(64)(mul)
#fc层
output=Dense(10,activation='relu')(mul)
model=Model(inputs=[inputs],outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 784)          615440      input_1[0][0]                    
__________________________________________________________________________________________________
multiply (Multiply)             (None, 784)          0           input_1[0][0]                    
                                                                 dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           50240       multiply[0][0]               

### 4.编译、训练

In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=32) 

Epoch 1/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1534 - accuracy: 0.7251
Epoch 2/100
1875/1875 [==============================] - 21s 11ms/step - loss: 0.1026 - accuracy: 0.8493
Epoch 3/100
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0854 - accuracy: 0.8803
Epoch 4/100
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0787 - accuracy: 0.9115
Epoch 5/100
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0822 - accuracy: 0.9050
Epoch 6/100
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0726 - accuracy: 0.9152
Epoch 7/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0714 - accuracy: 0.9172
Epoch 8/100
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0710 - accuracy: 0.9231 0s - loss: 0
Epoch 9/100
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0601 - accuracy: 0.9347
Epoch 10/100
1875/1875 [=====================

### 5.评估

In [6]:
model.evaluate(x_test, y_test, batch_size=32)

313/313 [==============================] - 2s 4ms/step - loss: 0.1001 - accuracy: 0.9501


[0.1000538244843483, 0.9501000046730042]

# 用自注意力机制处理IMDB影评数据

In [10]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences 
from keras.layers import Embedding ,Dense,Input,LSTM,Permute,Softmax,Lambda,Flatten, GRU
from keras import Model
from tensorflow.keras.utils import to_categorical

### 1.加载并格式化数据

In [11]:
max_len=200
(train_data, train_labels), (test_data, test_labels) = imdb.load_data()
train_data_pad = pad_sequences(train_data,padding="post",maxlen = max_len )
test_data_pad = pad_sequences(test_data,padding="post",maxlen = max_len )
train_labels_input = to_categorical(train_labels)
test_labels_input = to_categorical(test_labels)

17473536/17464789 [==============================] - 164s 9us/step


### 2.构建模型

In [13]:
input_=Input(shape=(max_len,))
words=Embedding(100000,64)(input_)
sen=GRU(64,return_sequences=True)(words)

#注意力层
attention_pre=Dense(64,name='attention_vec')(sen)
attention_probs=Softmax()(attention_pre)
attention_mul=Lambda(lambda x:x[0]*x[1])([attention_probs,sen])

output=Flatten()(attention_mul)
output=Dense(32,activation="relu")(output)
output = Dense(2, activation='softmax')(output)
model=Model(inputs=input_,outputs=output)

In [14]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["acc"])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 64)      6400000     input_3[0][0]                    
__________________________________________________________________________________________________
gru_1 (GRU)                     (None, 200, 64)      24960       embedding_1[0][0]                
__________________________________________________________________________________________________
attention_vec (Dense)           (None, 200, 64)      4160        gru_1[0][0]                      
____________________________________________________________________________________________

### 3.训练模型

In [15]:
model.fit(train_data_pad,train_labels_input,batch_size=64,epochs=5,
          validation_data=(test_data_pad,test_labels_input))

Epoch 1/5
391/391 [==============================] - 131s 330ms/step - loss: 0.4241 - acc: 0.7820 - val_loss: 0.2995 - val_acc: 0.8751
Epoch 2/5
391/391 [==============================] - 106s 270ms/step - loss: 0.1692 - acc: 0.9375 - val_loss: 0.3254 - val_acc: 0.8667
Epoch 3/5
391/391 [==============================] - 147s 377ms/step - loss: 0.0759 - acc: 0.9754 - val_loss: 0.4480 - val_acc: 0.8541
Epoch 4/5
391/391 [==============================] - 102s 261ms/step - loss: 0.0351 - acc: 0.9888 - val_loss: 0.5426 - val_acc: 0.8452
Epoch 5/5
391/391 [==============================] - 86s 220ms/step - loss: 0.0188 - acc: 0.9944 - val_loss: 0.7006 - val_acc: 0.8509


### 4.评估模型

In [16]:
test_loss, test_acc = model.evaluate(test_data_pad,test_labels_input,verbose=2) #损失值和准确率
print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

782/782 - 35s - loss: 0.7006 - acc: 0.8509

Test accuracy: 0.8509200215339661

Test loss: 0.7006161212921143
